# Demand forecasting using RNN with LSTM on PyTorch
In this tutorial, we will use the <a href="https://github.com/ray-project/ray_lightning">Ray Lightning plugin</a> (which runs on top <a href="https://docs.ray.io">Ray</a>) to speed up training and inference of Google's <a href="https://github.com/google-research/google-research/tree/master/tft">TemporalFusionTransformer</a> algorithm for RNN with LSTM, which has been adapted by <a href="https://pytorch-forecasting.readthedocs.io">PyTorch Forecasting</a>, which in turn is built on <a href="https://pytorch-lightning.readthedocs.io">PyTorch Lightning</a>. PyTorch Lightning is a set of APIs to simplify PyTorch, similar to the relationship of Keras to TensorFlow.

Ray can take any Python code and enable it to run distributed across multiple compute nodes.  The compute node cluster could be your own laptop cores or a cluster in any cloud.  Together with <a href="https://www.anyscale.com/">Anyscale cluster management</a>  for cloud, this is how Ray can speed up AI training and inferencing.

Demo data is NYC yellow taxi from: https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page 

Forecast goal:  Given 8 months historical taxi trips data for NYC, predict #pickups at each location, at an hourly granularity, for the next week.

Suggestion: Make a copy of this notebook. Make edits and run in the copied notebook. This way you will retain the original, executed notebook outputs.  

In [ ]:
###########
# Install libraries
###########

# !pip install ray     #install ray for the first time
# !pip install -U ray  #update ray to latest v1.9
# !pip install ray_lightning  #PyTorch Lightning plugin for Ray

# !pip install pytorch_lightning==1.4
# !pip install pytorch_forecasting

# Extra installs for Tensorboard to work with PyTorch on M1 apple
# conda uninstall -y tensorflow
# conda install -y tensorflow
# conda install pyparsing=2.4.2


In [1]:
###########
# Import libraries
###########

# Basic Python
import os
import warnings
warnings.filterwarnings("ignore")  # avoid printing out absolute paths
import fastparquet        # Engine for parquet support

# Open-source libraries:
import numpy as np
import pandas as pd
import ray                # Run distributed code
from ray.train import Trainer
from ray_lightning import RayPlugin

# PyTorch, PyTorch Lightning, and PyTorch Forecasting
import torch
import pytorch_lightning as pl
import pytorch_forecasting as ptf

# PyTorch visualization uses Tensorboard
import tensorflow as tf 
import tensorboard as tb 
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

!python --version
print(f"pytorch: {torch.__version__}")
print(f"pytorch_lightning: {pl.__version__}")
print(f"pytorch_forecasting: {ptf.__version__}")
print(f"ray: {ray.__version__}")


Python 3.8.12
pytorch: 1.10.0
pytorch_lightning: 1.4.0
pytorch_forecasting: 0.9.2
ray: 1.9.0


In [2]:
# Define functions
# Todo: Move functions inside util.py

# Convert data from pandas to PyTorch tensors.
def convert_pandas_pytorch_timeseriesdata(
    input_data_pandas_df, config):
    
    # specify data parameters
    FORECAST_HORIZON = config.get("forecast_horizon", 168)
    CONTEXT_LENGTH = config.get("context_length", 63)
    BATCH_SIZE = config.get("batch_size", 32)
    id_col_name = "pulocationid"
    target_value = "trip_quantity"
    covariates_numerical = ["time_idx", ]
                            # "mean_item_loc_weekday",
                            # "binned_max_item"]
    covariates_categorical=["day_hour"]
    
    the_df = input_data_pandas_df.copy()
    
    # define forecast horizon and training cutoff
    max_prediction_length = FORECAST_HORIZON  #decoder length = 1 week forecast horizon
    max_encoder_length = CONTEXT_LENGTH  # window or context length
    training_cutoff = the_df["time_idx"].max() - max_prediction_length

    # convert pandas to PyTorch tensor
    training_data = ptf.data.TimeSeriesDataSet(
        the_df[lambda x: x.time_idx <= training_cutoff],
        allow_missing_timesteps=True,
        time_idx="time_idx",
        target=target_value,
        group_ids=[id_col_name],
        min_encoder_length=5,  # allowing predictions without history
        max_encoder_length=max_encoder_length,
        min_prediction_length=1,
        max_prediction_length=max_prediction_length,
        static_categoricals=[id_col_name],
        # static_reals=["avg_population_2017", "avg_yearly_household_income_2017"],
        static_reals=[],
        time_varying_known_categoricals=covariates_categorical,
        # group of categorical variables can be treated as one variable
        # variable_groups={"special_days": special_days},  
        time_varying_known_reals=covariates_numerical,
        time_varying_unknown_categoricals=[],
        time_varying_unknown_reals=[target_value,],

        # https://pytorch-forecasting.readthedocs.io/en/v0.2.4/_modules/pytorch_forecasting/data.html
        target_normalizer=ptf.data.GroupNormalizer(
            groups=["pulocationid"], 
            transformation="softplus"  #forces positive values
        ), 
        add_relative_time_idx=True, # add as feature
        add_target_scales=True, # add as feature
        add_encoder_length=True, # add as feature
    )
    
    # create PyTorch dataloader for training
    train_loader = training_data\
                        .to_dataloader(
                            train=True, 
                            batch_size=BATCH_SIZE, 
                            num_workers=0)
    
    # create validation PyTorch data 
    # (predict=True) means make do inference using the validation data
    val_dataset = ptf.data.TimeSeriesDataSet\
                    .from_dataset(
                        training_data, 
                        df, predict=True, 
                        stop_randomization=True)

    # create PyTorch dataloaders for inference on validation data
    validation_loader = val_dataset\
                    .to_dataloader(
                        train=False, 
                        batch_size=BATCH_SIZE * 10, 
                        num_workers=0)
    
    # return original df converted to PyTorch tensors, and pytorch loaders
    return training_data, train_loader, validation_loader


# Define a PyTorch Lightning TemporalFusionTransformer model
def define_pytorch_model(train_dataset, config, ray_plugin):
    
    # get the parameters from config
    NUM_GPU = config.get("num_gpus", 0)
    EPOCHS = config.get("epochs", 30)
    LR = config.get("lr", 0.01)
    HIDDEN_SIZE = config.get("hidden_size", 40)
    HIDDEN_LAYERS = config.get("hidden_layers", 2)
    ATTENTION_HEAD_SIZE = config.get("attention_head_size", 4)
    HIDDEN_CONTINUOUS_SIZE = config.get("hidden_continuous_size", 1)
    
    print(f"learning_rate = {LR}")
    print(f"hidden_size = {HIDDEN_SIZE}")
    print(f"lstm_layers = {HIDDEN_LAYERS}")
    print(f"attention_head_size = {ATTENTION_HEAD_SIZE}")
    print(f"hidden_continuous_size = {HIDDEN_CONTINUOUS_SIZE}")

    # configure early stopping when validation loss does not improve 
    early_stop_callback = \
        pl.callbacks.EarlyStopping(
            monitor="val_loss", 
            min_delta=1e-4, 
            patience=10,   #1
            verbose=False, 
            mode="min")
    
    # configure logging
    lr_logger = pl.callbacks.LearningRateMonitor(logging_interval='epoch')
    logger = pl.loggers.TensorBoardLogger("lightning_logs")  # log results to a tensorboard

    # configure PyTorch trainer with Ray Lightning plugin
    torch_trainer = pl.Trainer(
        max_epochs=EPOCHS,
        gpus=NUM_GPU,
        # weights_summary="top",
        gradient_clip_val=0.1,
        limit_train_batches=30,  # running validation for every 30 batches
        # comment in to check that trainer dataset has no serious bugs
        # Note: No trainer checkpoints will be saved in fast mode
        # fast_dev_run=True,  
        callbacks=[lr_logger, early_stop_callback],
        logger=logger,
        
        # regular python - just comment out below line - runs fine!
        plugins=[ray_plugin]
    )
    print(f"checkpoints location: {torch_trainer.logger.log_dir}")

    # initialize the model
    tft = ptf.models.TemporalFusionTransformer.from_dataset(
        train_dataset,
        learning_rate=LR,
        hidden_size=HIDDEN_SIZE, #network size, bigger runs more slowly
        lstm_layers=HIDDEN_LAYERS, #hidden layers
        attention_head_size=ATTENTION_HEAD_SIZE,  #default 4 cells in LSTM layer
        # dropout=0.1,
        hidden_continuous_size=HIDDEN_CONTINUOUS_SIZE,  #similar to categorical embedding size
        output_size=7,  # 7 quantiles by default
        loss=ptf.metrics.QuantileLoss(),
        # # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
        log_interval=10,  
        reduce_on_plateau_patience=4, # reduce learning automatically
    )
    print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")
    
    # return the model and trainer
    return tft, torch_trainer


def train_func(config, ray_plugin):
    
    # read data into pandas dataframe
    filename = "../data/clean_taxi_hourly.parquet"
    df = pd.read_parquet(filename)
    df = df[["time_idx", "pulocationid", "day_hour",
                 "trip_quantity", "mean_item_loc_weekday",
                 "binned_max_item"]].copy()

    # convert data from pandas to PyTorch tensors
    train_dataset, train_loader, validation_loader = \
        convert_pandas_pytorch_timeseriesdata(df, config)

    # define a PyTorch deep learning forecasting model
    model, trainer  = define_pytorch_model(train_dataset, 
                                           config,
                                           ray_plugin)
    print(type(model))
    print(type(trainer))

    # now train the model
    trainer.fit(
        model,
        train_dataloaders=train_loader,
        val_dataloaders=validation_loader,
    )

    # return PyTorch DataLoader and Lightning Trainer
    return validation_loader, trainer


# Create and train a baseline model

In [3]:
# specify all the config parameters 
config = {"forecast_horizon": 168, "context_length": 63,
          "num_gpus":0, "batch_size": 128, "epochs": 2,
          "lr": 0.05, "hidden_size": 16, "hidden_layers": 2,
          "attention_head_size": 4, "hidden_continuous_size": 2}

# read data into pandas dataframe
filename = "~/Documents/AnyscaleDemosPrivate/demos/forecasting_demo/data/clean_taxi_hourly.parquet"
df = pd.read_parquet(filename)
df = df[["time_idx", "pulocationid", "day_hour",
             "trip_quantity", "mean_item_loc_weekday",
             "binned_max_item"]].copy()

# convert data from pandas to PyTorch tensors
print(f"Input data type: {type(df)}")
train_dataset, train_loader, validation_loader = \
    convert_pandas_pytorch_timeseriesdata(df, config)
print(f"Converted data type: {type(train_dataset)}")

# calculate baseline mean absolute error, i.e. predict next value as the last available value from the history
actuals = torch.cat(
            [
                y for x, (y, weight) in iter(validation_loader)
            ]
          )
baseline_predictions = ptf.models.Baseline().predict(validation_loader)


## EVALUATE THE BASELINE MODEL
# print MAE
(actuals - baseline_predictions).abs().mean()

#29.2463


Input data type: <class 'pandas.core.frame.DataFrame'>
Converted data type: <class 'pytorch_forecasting.data.timeseries.TimeSeriesDataSet'>


tensor(29.2463)

# Train a PyTorch Lightning DL Forecast Model

In [4]:
%%time

# specify all the config parameters 
config = {"forecast_horizon": 168, "context_length": 63,
          "batch_size": 128, "epochs": 2,
          "lr": 0.05, "hidden_size": 16, "hidden_layers": 2,
          "attention_head_size": 2, "hidden_continuous_size": 1} 

# Don't set ``gpus`` in the ``Trainer``.
# The actual number of GPUs is determined by ``num_workers``.
plugin = RayPlugin(num_workers=4, 
                   num_cpus_per_worker=1, 
                   use_gpu=False)
validation_loader, trainer = train_func(config, plugin)


print(type(validation_loader))
print(type(trainer))
print(type(plugin))

2021-12-13 18:25:05,260	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8266
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


learning_rate = 0.05
hidden_size = 16
lstm_layers = 2
attention_head_size = 2
hidden_continuous_size = 1
checkpoints location: lightning_logs/default/version_1
Number of parameters in network: 27.1k
<class 'pytorch_forecasting.models.temporal_fusion_transformer.TemporalFusionTransformer'>
<class 'pytorch_lightning.trainer.trainer.Trainer'>


(RayExecutor pid=41273) initializing ddp: GLOBAL_RANK: 0, MEMBER: 1/4
(RayExecutor pid=41274) initializing ddp: GLOBAL_RANK: 1, MEMBER: 2/4
(RayExecutor pid=41278) initializing ddp: GLOBAL_RANK: 2, MEMBER: 3/4
(RayExecutor pid=41271) initializing ddp: GLOBAL_RANK: 3, MEMBER: 4/4
(RayExecutor pid=41273) 
(RayExecutor pid=41273)    | Name                               | Type                            | Params
(RayExecutor pid=41273) ----------------------------------------------------------------------------------------
(RayExecutor pid=41273) 0  | loss                               | QuantileLoss                    | 0     
(RayExecutor pid=41273) 1  | logging_metrics                    | ModuleList                      | 0     
(RayExecutor pid=41273) 2  | input_embeddings                   | MultiEmbedding                  | 6.8 K 
(RayExecutor pid=41273) 3  | prescalers                         | ModuleDict                      | 12    
(RayExecutor pid=41273) 4  | static_variable_se

Validation sanity check:   0%|          | 0/1 [00:00<?, ?it/s]


(RayExecutor pid=41274) /Users/christy/mambaforge/envs/ray/lib/python3.8/site-packages/pytorch_forecasting/data/timeseries.py:1657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
(RayExecutor pid=41274)   target_scale = torch.tensor([batch[0]["target_scale"] for batch in batches], dtype=torch.float)
(RayExecutor pid=41278) /Users/christy/mambaforge/envs/ray/lib/python3.8/site-packages/pytorch_forecasting/data/timeseries.py:1657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
(RayExecutor pid=41278)   target_scale = torch.tensor([batch[0]["target_scale"] for batch in bat

Epoch 0:   0%|          | 0/31 [00:00<00:00, 6626.07it/s]             


(RayExecutor pid=41273) /Users/christy/mambaforge/envs/ray/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
(RayExecutor pid=41273)   rank_zero_warn(
(RayExecutor pid=41273) /Users/christy/mambaforge/envs/ray/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:322: UserWarning: The number of training samples (30) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
(RayExecutor pid=41273)   rank_zero_warn(
(RayExecutor pid=41278) [W reducer.cpp:1303] Warning: find_unused_parameters=True was specified in DDP constructor, but did not find any unused parameters in the forward pass. This

RayTaskError(AttributeError): [36mray::RayExecutor.execute()[39m (pid=41278, ip=127.0.0.1, repr=<ray_lightning.ray_ddp.RayExecutor object at 0x13546a3a0>)
  File "/Users/christy/mambaforge/envs/ray/lib/python3.8/site-packages/ray_lightning/ray_ddp.py", line 54, in execute
    return fn(*args, **kwargs)
  File "/Users/christy/mambaforge/envs/ray/lib/python3.8/site-packages/ray_lightning/ray_ddp.py", line 297, in execute_remote
    super(RayPlugin, self).new_process(
  File "/Users/christy/mambaforge/envs/ray/lib/python3.8/site-packages/pytorch_lightning/plugins/training_type/ddp_spawn.py", line 201, in new_process
    results = trainer.run_stage()
  File "/Users/christy/mambaforge/envs/ray/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py", line 996, in run_stage
    return self._run_train()
  File "/Users/christy/mambaforge/envs/ray/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py", line 1045, in _run_train
    self.fit_loop.run()
  File "/Users/christy

2021-12-13 18:25:23,371	ERROR worker.py:84 -- Unhandled error (suppress with RAY_IGNORE_UNHANDLED_ERRORS=1): ray::RayExecutor.execute() (pid=41274, ip=127.0.0.1, repr=<ray_lightning.ray_ddp.RayExecutor object at 0x11654a3a0>)
  File "/Users/christy/mambaforge/envs/ray/lib/python3.8/site-packages/ray_lightning/ray_ddp.py", line 54, in execute
    return fn(*args, **kwargs)
  File "/Users/christy/mambaforge/envs/ray/lib/python3.8/site-packages/ray_lightning/ray_ddp.py", line 297, in execute_remote
    super(RayPlugin, self).new_process(
  File "/Users/christy/mambaforge/envs/ray/lib/python3.8/site-packages/pytorch_lightning/plugins/training_type/ddp_spawn.py", line 201, in new_process
    results = trainer.run_stage()
  File "/Users/christy/mambaforge/envs/ray/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py", line 996, in run_stage
    return self._run_train()
  File "/Users/christy/mambaforge/envs/ray/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py", li

# Evaluate Performance

In [ ]:
## EVALUATE THE DL MODEL

# load the best model according to the validation loss (given that
# we use early stopping, this is not necessarily the last epoch)
best_model_path = trainer.checkpoint_callback.best_model_path
print(best_model_path)
best_tft = ptf.models.TemporalFusionTransformer.load_from_checkpoint(best_model_path)

# calcualte mean absolute error on validation set
actuals = torch.cat([y[0] for x, y in iter(validation_loader)])
predictions = best_tft.predict(validation_loader)

# print MAE
(actuals - predictions).abs().mean()

#18.9355 with  "attention_head_size": 2, "hidden_continuous_size": 1} 

In [ ]:
# Visualize in tensorboard, hit ctrl-c when done
!tensorboard --logdir=lightning_logs --load_fast=false

# Plot actuals vs predictions

In [ ]:
# raw predictions are a dictionary from which all kind of information including quantiles can be extracted
raw_predictions, x = best_tft.predict(validation_loader, mode="raw", return_x=True)

for idx in range(5):  # plot 5 examples
    best_tft.plot_prediction(x, raw_predictions, idx=idx, add_loss_to_title=True);